In [ ]:
import polars as pl
from music.api import get_playlist_tracks, get_user_client, get_user_playlists

# Specify playlists to keep; i.e. consider for labels
KEEP_PLAYLISTS = {
    "Alternative",
    "Ambient",
    "Angstry Rock",
    "Bluegrass",
    "Chill",
    "Chill EDM",
    "Country",
    "EDM",
    "Feels",
    # 'Good Vibes',
    # 'Groovy',
    "Hip Hop",
    "Indie",
    "Classic Rock",
    "Pop",
    "Punk",
    # 'Soundtracks',
    # 'Throwbacks',
}

# Get Data

In [ ]:
# Get client
client = get_user_client()

In [ ]:
# Get playlists
playlist_objs = get_user_playlists(client)
playlists = pl.DataFrame(playlist_objs)[["id", "name"]]

In [ ]:
# Print playlists
print(playlists["name"].to_list())

In [ ]:
def get_tracks(playlist: pl.DataFrame) -> pl.DataFrame:
    # Get info
    id, name = playlist.row(0)
    tracks_dicts = get_playlist_tracks(client, id)

    # Finalize output
    out = pl.DataFrame({"track_id": t["track"]["id"], "track_name": t["track"]["name"]} for t in tracks_dicts)
    out = out.with_columns(playlist_id=pl.lit(id), playlist_name=pl.lit(name))

    return out


# Get tracks from playlists
to_get = playlists.filter(pl.col("name").is_in(KEEP_PLAYLISTS))
df = to_get.group_by("id").map_groups(get_tracks)

In [ ]:
# Save data
df.write_parquet("playlists.parquet")